# 2에서 바뀐 부분

augmentation 삭제, efficientnet 대신 mobilenet 이용

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
label_mapping = {
    '000': 0,
    '001': 1,
    '010': 2,
    '011': 3,
    '100': 4,
    '101': 5,
    '110': 6,
    '111': 7
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [ ]:
df = pd.read_excel('label2.xlsx')
df = df[~df['label'].str.contains("-")]  # Filter out invalid entries
df['label'] = df['label'].map(label_mapping)  # Map string labels to integers

In [ ]:
print(df)

      File Number life head  heart  label
0               1    0    0      0      0
1               2    0    0      0      0
2               5    1    1      0      6
3              12    0    0      1      1
4              13    1    1      0      6
...           ...  ...  ...    ...    ...
1030         5353    0    0      1      1
1031         5354    1    0      1      5
1032         5366    1    0      0      4
1033         5367    0    0      0      0
1035         5385    0    0      1      1

[965 rows x 5 columns]


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def load_and_preprocess_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./Mask', f"image{row['File Number']}.png")
        image = Image.open(image_path).resize((128, 128))
        image = np.array(image) / 255.0
        if image.ndim == 2 or image.shape[2] == 1:
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB
        images.append(image)
    return np.array(images)

In [ ]:
train_images = load_and_preprocess_images(train_df)
test_images = load_and_preprocess_images(test_df)

  0%|          | 0/772 [00:00<?, ?it/s]

  0%|          | 0/193 [00:00<?, ?it/s]

In [ ]:
train_labels = to_categorical(train_df['label'], num_classes=8)
test_labels = to_categorical(test_df['label'], num_classes=8)

In [ ]:
# Initialize MobileNetV2 model with pretrained ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model

# Create new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')  # 8 classes for the final layer
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True)
]

history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # May increase if model is not overfitting
    callbacks=callbacks
)

# Load the best weights and unfreeze the base_model for fine-tuning
model.load_weights('best_model.h5')
base_model.trainable = True

Epoch 1/50
25/25 [==============================] - 8s 247ms/step - loss: 2.0718 - accuracy: 0.2552 - val_loss: 1.8155 - val_accuracy: 0.3109
Epoch 2/50
25/25 [==============================] - 7s 272ms/step - loss: 1.7753 - accuracy: 0.3238 - val_loss: 1.6788 - val_accuracy: 0.4249
Epoch 3/50
25/25 [==============================] - 6s 234ms/step - loss: 1.6509 - accuracy: 0.3510 - val_loss: 1.6322 - val_accuracy: 0.3782
Epoch 4/50
25/25 [==============================] - 7s 271ms/step - loss: 1.5843 - accuracy: 0.4080 - val_loss: 1.6099 - val_accuracy: 0.3886
Epoch 5/50
25/25 [==============================] - 6s 226ms/step - loss: 1.5454 - accuracy: 0.4158 - val_loss: 1.5682 - val_accuracy: 0.4352
Epoch 6/50
25/25 [==============================] - 5s 218ms/step - loss: 1.5350 - accuracy: 0.4223 - val_loss: 1.5642 - val_accuracy: 0.4197
Epoch 7/50
25/25 [==============================] - 7s 268ms/step - loss: 1.4836 - accuracy: 0.4391 - val_loss: 1.6109 - val_accuracy: 0.3990
Epoch 

In [ ]:
# Recompile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

7/7 [==============================] - 2s 152ms/step - loss: 1.4199 - accuracy: 0.4767
Test accuracy: 47.67%


In [ ]:
# Fine-tune the model
history_fine = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=20,  # Additional epochs for fine-tuning
    callbacks=callbacks
)

Epoch 1/20
25/25 [==============================] - 35s 944ms/step - loss: 3.5042 - accuracy: 0.2137 - val_loss: 1.4394 - val_accuracy: 0.4611
Epoch 2/20
25/25 [==============================] - 24s 959ms/step - loss: 3.0708 - accuracy: 0.2358 - val_loss: 1.5024 - val_accuracy: 0.4249
Epoch 3/20
25/25 [==============================] - 23s 925ms/step - loss: 2.7019 - accuracy: 0.2746 - val_loss: 1.5617 - val_accuracy: 0.4093
Epoch 4/20
25/25 [==============================] - 23s 926ms/step - loss: 2.5802 - accuracy: 0.2759 - val_loss: 1.6590 - val_accuracy: 0.3627
Epoch 5/20
25/25 [==============================] - 21s 855ms/step - loss: 2.4177 - accuracy: 0.2979 - val_loss: 1.7439 - val_accuracy: 0.3627
Epoch 6/20
25/25 [==============================] - 23s 909ms/step - loss: 2.2091 - accuracy: 0.3187 - val_loss: 1.8333 - val_accuracy: 0.3523
Epoch 6: early stopping


In [ ]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

7/7 [==============================] - 1s 135ms/step - loss: 1.8333 - accuracy: 0.3523
Test accuracy: 35.23%
